In [53]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [54]:
pip install MLNN

In [55]:
from MLNN import*

In [56]:
import LLC_4_1 as LLC
# import https://github.com/vladgap/LLC/blob/main/LLC_4_1.py?raw=true as LLC

In [57]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_lit1
lit data with NN model (7 hidden sigmoids)

### Equilibrium literature data

In [ ]:
data1=pd.DataFrame(data=[
[63.5,	1.3,	42.4,	15.0],
[58.7,	1.2,	36.9,	13.8],
[52.0, 1.0,	27.7,	11.8],
[45.5,	1.0,	18.9,	9.6],
[39.1,	1.4,	10.2,	8.3],
[31.5,	1.5,	5.1,	8.8],
[22.9,	2.0,	2.3,	9.0],
[12.6,	2.2,	0.90,	9.3],
[6.3,	2.4,	0.45,	9.45],
[0.01,	2.6,	0.01,	9.6],
],
columns=['H3PO4_aq', 'solvent_aq', 'H3PO4_org', 'water_org'])

In [ ]:
data1.head()

,H3PO4_aq,solvent_aq,H3PO4_org,water_org
0,63.5,1.3,42.4,15.0
1,58.7,1.2,36.9,13.8
2,52.0,1.0,27.7,11.8
3,45.5,1.0,18.9,9.6
4,39.1,1.4,10.2,8.3


In [ ]:
fig1=go.Figure()
[fig1.add_trace(go.Scatter(x=data1.H3PO4_aq, y=data1[data1.columns[i]], name=data1.columns[i])) for i in range(1,4)]
fig1.update_layout(title='', width=1200, height=600,)
fig1.show()

In [ ]:
X1=np.array(data1)[:,[0]]
T1=np.array(data1)[:,[1,2,3]]

### Model
"Model for Rotem LLC simulation 1"

In [ ]:
scaler_X1 = preprocessing.StandardScaler()
scaler_T1 = preprocessing.StandardScaler()
scaler_X1.mean_, scaler_X1.scale_ = [33.211], [21.13387066]
scaler_T1.mean_, scaler_T1.scale_ = [1.66,  14.486, 10.465], [0.56071383, 15.23739033,  2.16703138]

bp1=VectorBackProp(layers=[1,7,3], hidden_activation = 'sigmoid')
manual_best_weights_for_now2=[[[-4.215621245080982,
   -8.235086763118087,
   -1.6251342150606451,
   -6.9206357615871035,
   -3.4092427169369826,
   -7.4263402914028775,
   -3.7003458539301977]],
 [[0.8764395040443358,
   3.9145850159876945,
   -1.5013585830931442,
   -1.194371163565281,
   4.118411550251139,
   5.892242255497829,
   -6.755593628941706]],
 [[-2.6609909047049918, -0.44480776958732143, 1.9256391063013463],
  [2.566022222516618, -0.5042582093932455, -1.432212975592254],
  [1.405004033981333, -0.2988952466228391, 0.5807591718257896],
  [2.0692859230668117, -0.0780871568942863, -0.5145373287488922],
  [-0.7989625307881565, -1.957459884661486, -2.8991426864622065],
  [-0.5821795598385298, -0.22284877970127598, -0.9077232625087237],
  [1.619488070885094, 0.03697105802710269, 0.008613201857731975]],
 [[-0.4010006364699687, 2.4687577407660806, 2.995012357373033]]]
bp1.import_weights(manual_best_weights_for_now2)

In [ ]:
class EQUIL_rotem_lit1: # literature data for pentanol 25C
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        # the model predicts %solvent_aq, %H3PO4_aq, %water_org from %H3PO4_aq
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        pred=scaler_T1.inverse_transform(bp1.run(scaler_X1.transform(x[:,[1]])))
        # prediction is column vector of lists [%solvent_aq, %H3PO4_aq, %water_org]

        self.xout = np.array([pred[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([pred[:,2], pred[:,1]]).T # [[%water_org, %H3PO4_org], ...]

## EQUIL_rotem_washing1
Rotem solvent, estimation by polynoms

In [ ]:
class EQUIL_rotem_washing1: # prepared using purified and washed solvents
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        y=np.zeros_like(x)
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # y = A + B*x + C*x^2 + D*x^3

        # %solvent_aq
        D,C,B,A = -3.78E-06,8.15E-04,- 5.73E-02,2.45E+00
        x[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %water_org
        D,C,B,A = 4.45E-05,- 1.56E-03,- 1.84E-02,7.58E+00
        y[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %H3PO4_org
        # D,C,B,A = 2.00E-04,- 4.86E-03,7.07E-02,2.73E-02
        # y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3
        G,F,E,D,C,B,A = -1.97E-09,2.02E-07,- 2.95E-06,- 3.83E-05,2.60E-03,2.38E-02,1.70E-02
        y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3 + E*x[:,1]**4 + F*x[:,1]**5 + G*x[:,1]**6

        self.xout = np.array([x[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([y[:,0], y[:,1]]).T # [[%water_org, %H3PO4_org], ...]

In [ ]:
# EQUIL_rotem_washing1([[2,20],[1.5,30],[1.3,40]]).xout

## EQUIL_rotem_washing2
Rotem solvent, full profile, sulfate

In [ ]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org
4.15	0.147	0.0733	9.6	2.5	0.0626
14.9	0.0806	0.359	9.5	2.2	0.0852
28.6	0.0329	1.68	9.3	1.9	0.0787
45.7	0.0106	8.31	8.5	1.6	0.0664
48.2	0.0078	10.7	8.5	1.3	0.0631
49.4	0.0058	12.2	8.7	1.0	0.0641
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data2=pd.DataFrame(data[1:], columns=data[0])
data2

,H3PO4_aq,SO4_aq,H3PO4_org,water_org,solvent_aq,SO4_org
0,4.15,0.1470,0.0733,9.6,2.5,0.0626
1,14.90,0.0806,0.3590,9.5,2.2,0.0852
2,28.60,0.0329,1.6800,9.3,1.9,0.0787
3,45.70,0.0106,8.3100,8.5,1.6,0.0664
4,48.20,0.0078,10.7000,8.5,1.3,0.0631
5,49.40,0.0058,12.2000,8.7,1.0,0.0641


In [ ]:
X2=np.array(data2)[:,[0,1]]
T2_1=np.array(data2)[:,[2]]
T2_2=np.array(data2)[:,[3]]
T2_3=np.array(data2)[:,[4]]
T2_4=np.array(data2)[:,[5]]

scaler_X2 = preprocessing.StandardScaler().fit(X2)
scaler_T2_1 = preprocessing.StandardScaler().fit(T2_1)
scaler_T2_2 = preprocessing.StandardScaler().fit(T2_2)
scaler_T2_3 = preprocessing.StandardScaler().fit(T2_3)
scaler_T2_4 = preprocessing.StandardScaler().fit(T2_4)

X_sc2 = scaler_X2.transform(X2)
# T_sc2_1 = scaler_T2_1.transform(T2_1)
# T_sc2_2 = scaler_T2_1.transform(T2_2)
# T_sc2_3 = scaler_T2_1.transform(T2_3)
# T_sc2_4 = scaler_T2_1.transform(T2_4)

In [ ]:
class EQUIL_rotem_washing2: # from full profile, sulfate
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp2_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_1=[[[0.1735491429461148, 1.8140938189194906],
  [-0.6530409129171293, -0.6441814583921941]],
 [[0.7411897128053353, -1.2033881701371891]],
 [[0.3516552639162472], [1.6547118904069311]],
 [[-1.0860308848681413]]]
        bp2_1.import_weights(manual_weights2_1)
        predics2_1=abs(scaler_T2_1.inverse_transform(bp2_1.run(scaler_X2.transform(x[:,[1,2]]))))

        # water_org
        bp2_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_2=[[[-1.356083366626272, 2.1006456906712234],
  [-0.6309805208401018, -0.5374527149916659]],
 [[1.0515374304634164, -2.3182953246087115]],
 [[1.4841383017903245], [3.153523627853289]],
 [[-1.7074908699001965]]]
        bp2_2.import_weights(manual_weights2_2)
        predics2_2=abs(scaler_T2_2.inverse_transform(bp2_2.run(scaler_X2.transform(x[:,[1,2]]))))

        # solvent_aq
        bp2_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_3=[[[2.0994823528593702, -1.2293957752203057],
  [-0.6972915446623457, 0.5216771334283966]],
 [[-2.3459563115354527, 1.775004898861346]],
 [[-3.1726827033207408], [0.4025651395234753]],
 [[-0.4638947231253712]]]
        bp2_3.import_weights(manual_weights2_3)
        predics2_3=abs(scaler_T2_3.inverse_transform(bp2_3.run(scaler_X2.transform(x[:,[1,2]]))))

        # SO4_org
        bp2_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_4=[[[-2.7308241499437758, 1.8373933992350313],
  [-2.021898044039464, -0.8845457922101067]],
 [[1.0694417308325381, 0.08920200363835196]],
 [[2.7835653102552724], [1.6175935763910405]],
 [[-4.969235753353272]]]
        bp2_4.import_weights(manual_weights2_4)
        predics2_4=abs(scaler_T2_4.inverse_transform(bp2_4.run(scaler_X2.transform(x[:,[1,2]]))))

        self.xout = np.array([predics2_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2_2[:,0], predics2_1[:,0], predics2_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        print (self.yout, self.xout)

## EQUIL_rotem_washing3
ניסויי הפרדה עם ממס מטוהר

In [58]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org
0.011	0.0932	0.072	12.09	1.70	0.0186
10.325	0.0017	0.229	10.66	1.52	0.02
10.380	0.0990	0.269	10.41	1.50	0.0208
19.821	0.0031	0.635	11.3	1.34	0.01884
19.710	0.0928	0.773	12.34	1.32	0.0195
28.710	0.0057	1.435	11.96	1.28	0.01978
27.882	0.0858	1.325	11.1	1.34	0.0222
35.611	0.0071	2.871	11.94	1.28	0.036
35.887	0.0846	2.981	12.16	1.24	0.0374
43.065	0.0082	3.561	12.12	1.28	0.057
43.065	0.0802	5.604	11.7	1.06	0.0608
49.966	0.0074	8.723	12.28	1.28	0.0872
0.017	0.1540	0.042	13.28	1.46	0.0164
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
data3_names=CopyPasteToArray('''
w12	w21	w22	w31	w32	w41	w42	w51	w52	w61	w62	w71	w13
''')

In [59]:
# X3=np.array(data3)[:,[0,1]]
# T3_1=np.array(data3)[:,[2]]
# T3_2=np.array(data3)[:,[3]]
# T3_3=np.array(data3)[:,[4]]
# T3_4=np.array(data3)[:,[5]]

# scaler_X3 = preprocessing.StandardScaler().fit(X3)
# scaler_T3_1 = preprocessing.StandardScaler().fit(T3_1)
# scaler_T3_2 = preprocessing.StandardScaler().fit(T3_2)
# scaler_T3_3 = preprocessing.StandardScaler().fit(T3_3)
# scaler_T3_4 = preprocessing.StandardScaler().fit(T3_4)

# X_sc3 = scaler_X3.transform(X3)

In [60]:
Xscale=[24.95769231,  0.0556    ] ,[15.83973345 , 0.04945522]
Tscale=[ 2.19384615 ,11.79538462 , 1.35384615 , 0.03342462] ,[2.47579249, 0.73668295 ,0.15259491 ,0.02109944]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_T3_1.mean, scaler_T3_1.scale_=Tscale[0][0], Tscale[1][0]
scaler_T3_2.mean, scaler_T3_2.scale_=Tscale[0][1], Tscale[1][1]
scaler_T3_3.mean, scaler_T3_3.scale_=Tscale[0][2], Tscale[1][2]
scaler_T3_4.mean, scaler_T3_4.scale_=Tscale[0][3], Tscale[1][3]

In [61]:
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[1.3378100076012602, -0.23961073392218685],
  [0.04604608999058306, -0.01362935480464816]],
 [[-0.13967681304065602, 0.3658030173490756]],
 [[1.3443495997514352], [-0.4211945072460851]],
 [[-0.5176765751423951]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[-0.003176245086173946, 0.922720194179483],
  [0.025836730938298032, -0.29271027455453025]],
 [[-0.162875602876959, 0.2764312267422203]],
 [[0.02925249535375607], [0.7985118883946332]],
 [[-0.4749218275525917]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3.transform(x[:,[1,2]]))))

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[0.044601620561964116, 9.065855748211808],
  [0.018485353223382184, 0.09664600884033867]],
 [[-0.1299251444151146, -3.493201184453457]],
 [[0.0699275541320634], [0.2892791954157]],
 [[-0.6633848526245624]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Literature data

### 1 - Washing



In [ ]:
bat1_2=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[5,12], xin=[5,0], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])
# bat1_2=LLC.Battery(4, 87.6, 112.4, [0, 0], [0, 32.56], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=.7, entrainment_perc_out=10)

In [ ]:
bat1_2.runs

35

In [ ]:
table1_2=LLC.BatteryTableFull(bat1_2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,89.055359,85.475686,83.948464,83.262505
"y1 in, %",5.000000,8.829622,9.079322,9.354564,9.494674
"y2 in, %",12.000000,6.974399,3.352100,1.529652,0.668417
"Aq in, ton/hr",34.382667,30.833850,29.321083,28.640994,29.000000
"x1 in, %",2.075036,2.268582,2.424578,2.529920,5.000000
"x2 in, %",19.948914,10.447329,4.783224,1.865826,0.000000
"Org out, ton/hr",89.055213,85.474791,83.948421,83.262549,83.906581
"y1 out, %",8.829647,9.079543,9.354583,9.494752,9.558549
"y2 out, %",6.974221,3.350772,1.529553,0.667935,0.281837
"Aq out, ton/hr",34.452985,34.379990,30.833663,29.320314,28.640963


In [ ]:
bat=bat1_2
fig1_3 = go.Figure()
n=1 #H3PO4
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig1_3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig1_3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig1_3.update_layout(title='', width=1200, height=600,)
fig1_3.show()

In [ ]:
fig1_4=go.Figure()
current_data = data1
current_table = table1_2
[fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig1_4.update_xaxes(title_text="x, %", range=[20, 40])
# fig1_4.update_yaxes(title_text="y, %", range=[0, 15])
fig1_4.update_layout(title='', width=1200, height=600,)
fig1_4.show()

### 2 - Extraction

In [ ]:
bat2=LLC.Battery(stages_num=7, Oin=90, Ain=29, yin=[5,0], xin=[0,40], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[[.8, 1], [0.5, .8], [.6,.9]])

In [ ]:
bat2.runs

38

In [ ]:
bat2.eff

array([[1. , 1. ],
       [1. , 0.8],
       [1. , 0.9],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ]])

In [ ]:
table2=LLC.BatteryTableFull(bat2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",90.000000,96.473211,96.995114,97.591662,98.289728,99.065594,100.080908
"y1 in, %",5.000000,8.991353,8.936158,8.913425,8.882908,8.818593,8.686241
"y2 in, %",0.000000,2.343352,2.895783,3.477733,4.154708,4.927815,5.937057
"Aq in, ton/hr",24.303520,24.832050,25.420661,26.121458,26.893674,27.928541,29.000000
"x1 in, %",1.846572,1.742477,1.628782,1.519116,1.440582,1.414597,0.000000
"x2 in, %",26.076255,27.717279,29.376141,31.174464,33.199446,35.744126,40.000000
"Org out, ton/hr",96.473586,96.992375,97.593864,98.283709,99.070893,100.077211,101.186610
"y1 out, %",8.991297,8.936323,8.913326,8.883276,8.818035,8.687748,8.456033
"y2 out, %",2.343821,2.892832,3.479975,4.148719,4.933070,5.927017,7.477362
"Aq out, ton/hr",17.829934,24.312885,24.821911,25.429411,26.112510,26.916924,27.894298


In [ ]:
bat=bat2
fig2_1 = go.Figure()
n=1 #H3PO4
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig2_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig2_1.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig2_1.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig2_1.update_layout(title='', width=1200, height=600,)
fig2_1.show()

In [ ]:
fig2_2=go.Figure()
current_data = data1
current_table = table2
[fig2_2.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig2_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig2_2.update_yaxes(title_text="y, %", range=[0, 15])
fig2_2.update_layout(title='', width=1200, height=600,)
fig2_2.show()

## Washing data
Rotem solvent, estimation by polynoms

### 3 - Washing1

In [ ]:
bat3=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1)#, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])

KeyboardInterrupt: ignored

In [ ]:
bat3.runs

In [ ]:
table3=LLC.BatteryTable(bat3).data

In [ ]:
bat=bat3
fig3 = go.Figure()
n=1 #H3PO4
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig3.update_layout(title='', width=1200, height=600,)
fig3.show()

In [ ]:
fig3_2=go.Figure()
# current_data = data1
current_table = table3
# [fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig3_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig3_2.update_yaxes(title_text="y, %", range=[0, 15])
fig3_2.update_layout(title='', width=1200, height=600,)
fig3_2.show()

### 4 - Washing2

In [ ]:
bat4=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1, convergence=5e-4, eff=[1, 0.9], entrainment_perc_in=[2,0], entrainment_comp_in=[[5,5],[0,0]], entrainment_perc_out=[3,4])

In [ ]:
bat4.runs

In [ ]:
table4=LLC.BatteryTableFull(bat4).data

In [ ]:
bat=bat4
fig4 = go.Figure()
n=1 #H3PO4
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig4.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig4.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig4.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig4.update_layout(title='Operation Data', width=1200, height=600,)
fig4.show()

In [ ]:
fig4_2=go.Figure()
# current_data = data1
current_table = table4
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig4_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig4_2.update_yaxes(title_text="y, %", range=[0, 15])
fig4_2.update_layout(title='Equilibrium Data', width=1200, height=600,)
fig4_2.show()

## Rotem washing with sulfate

### 5 - Washing1

In [ ]:
EQUIL_rotem_washing2([[2.2,14.9,.0806]]).yout

[[9.56542351 0.35871824 0.08520132]] [[ 2.2011776 14.9        0.0806   ]]


array([[9.56542351, 0.35871824, 0.08520132]])

In [ ]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [2, 0.1, .02], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

[[9.43019245 0.20932071 0.07283325]] [[ 2.19818904 16.21426609  0.08833174]]
[[9.30172822 1.87808801 0.08863289]] [[1.87760943e+00 2.88068189e+01 2.42557218e-02]]
[[9.67842195 2.57175191 0.13196243]] [[ 1.83723689e+00  2.68844615e+01 -8.22752546e-03]]
[[9.40774599 1.41584489 0.08787853]] [[ 1.98162    24.14670566  0.0410468 ]]
[[8.74299355 0.09728085 0.0266234 ]] [[ 2.09270285 26.11681351  0.1073246 ]]
[[ 8.71494393 10.32003728  0.05946183]] [[ 1.11956982 54.77546413  0.05751555]]
[[8.70497911 0.27831522 0.02663159]] [[ 2.02946256 28.71408958  0.0954719 ]]
[[8.73139876 0.33424516 0.02663486]] [[ 2.02535362 28.62486958  0.09290654]]
[[8.61874053 0.09909448 0.02661727]] [[ 2.06229808 28.3118691   0.1082116 ]]
[[8.628167   0.09852728 0.02661639]] [[ 2.07180798 27.88816926  0.10976343]]
[[8.60548055 0.09880625 0.0266151 ]] [[ 2.06713123 28.25772227  0.11012198]]
[[8.60813102 0.09878081 0.02661528]] [[ 2.06755767 28.21882933  0.11005313]]
[[8.60822308 0.09877921 0.02661528]] [[ 2.06758444 2

In [ ]:
test1.runs

10

In [ ]:
test1.output()

Oout= 74.5042053393592
yout= [8.60822308 0.09877921 0.02661528]
yout_tag= [8.60822308 0.09877921 0.02661528]  %water_org_equilib, %solubles_equilib
Aout= 44.49579466064079
xout= [ 2.06758444 28.21705147  0.11005343]
equilibrium efficiency= [1. 1. 1.]
error= [4.11296949e-05 5.50745531e-05 6.80630206e-05]
convergence= [-0.00040428 -0.00022316 -0.00016548]


In [ ]:
test1 = LLC.Stage(90, 29, [8.60822308, 0.09877921, 0.02661528], [2.06758444, 28.21705147, 0.11005343], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

[[9.40587212 0.08828243 0.06697643]] [[ 2.24355622 14.81172832  0.10004056]]
[[9.64682125 2.83556223 0.13886871]] [[ 1.78411875e+00  2.90630819e+01 -1.82035537e-02]]
[[8.21751925 0.08034712 0.02647194]] [[ 2.37666577 25.232862    0.23555299]]
[[8.21268012 0.05489456 0.02621698]] [[ 2.80349145 26.46953754  0.44614586]]
[[8.74664907 0.11174503 0.02662841]] [[ 2.06769155 26.97713024  0.10147547]]
[[8.60768029 0.09853519 0.02661446]] [[ 2.07167602 28.0758201   0.11100685]]
[[8.60564232 0.09871973 0.02661485]] [[ 2.06858194 28.20448409  0.11043802]]
[[8.60879365 0.09878278 0.02661534]] [[ 2.06752461 28.2140926   0.11000452]]
[[8.60794753 0.09877988 0.02661526]] [[ 2.06757328 28.21991049  0.11006804]]


In [ ]:
bat5_1=LLC.Battery(stages_num=4, Oin=90, Ain=29, yin=[8.60822308, 1, 0.05], xin=[2.06758444, 30, 0.3], EQUIL=EQUIL_rotem_washing2)

[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]
[[8.21542165 0.07060956 0.02631295]] [[ 2.53998164 31.47867061  0.35630376]]
[[8.21539376 0.07041787 0.02631317]] [[ 2.54319538 31.28743445  0.35649378]]
[[8.21539538 0.07041956 0.02631351]] [[ 2.543167   31.25688796  0.35626778]]
[[8.21539487 0.07041688 0.02631348]] [[ 2.54321195 31.25704348  0.35629013]]
[[8.21539496 0.07041739 0.02631349]] [[ 2.54320338 31.25703412  0.35628601]]
[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]
[[8.21542165 0.07060956 0.02631295]] [[ 2.53998164 31.47867061  0.35630376]]
[[8.21539376 0.07041787 0.02631317]] [[ 2.54319538 31.28743445  0.35649378]]
[[8.21539538 0.07041956 0.02631351]] [[ 2.543167   31.25688796  0.35626778]]
[[8.21539487 0.07041688 0.02631348]] [[ 2.54321195 31.25704348  0.35629013]]
[[8.21539496 0.07041739 0.02631349]] [[ 2.54320338 31.25703412  0.35628601]]
[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]

In [ ]:
bat5_1.runs

6

In [ ]:
table5=LLC.BatteryTable(bat5_1).data

,Stage 1,Stage 2,Stage 3,Stage 4
"Org in, ton/hr",90.000000,88.607990,88.615281,88.615270
"y1 in, %",8.608223,8.215397,8.216589,8.216587
"y2 in, %",1.000000,0.070425,0.076451,0.076442
"y3 in, %",0.050000,0.026314,0.026386,0.026385
"Aq in, ton/hr",29.114540,29.121741,29.121729,29.000000
"x1 in, %",2.442020,2.442172,2.442171,2.067584
"x2 in, %",29.864179,29.874934,29.874918,30.000000
"x3 in, %",0.298710,0.298857,0.298856,0.300000
"Org out, ton/hr",88.607990,88.615281,88.615270,88.493540
"y1 out, %",8.215397,8.216589,8.216587,8.216587


In [ ]:
bat=bat5_1
fig5 = go.Figure()
n=1 #H3PO4
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig5.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig5.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig5.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig5.update_layout(title='Operation Data', width=1200, height=600,)
fig5.show()

In [ ]:
fig_equil2 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.SO4_org, mode='markers', name='SO4_org'), 2, 2)

bat=bat5_1
# H3PO4_org
fig_equil2.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', name='Efficiency'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])), y=np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])), z=np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])),
                                mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),
                                                                                                                            np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])),
                                                                                                                            np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])))],
                                textposition ='bottom center', name='Operation', marker_color='green'),1,1)

fig_equil2.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil2.update_layout(
    scene1_zaxis_range=[0,15],
    scene2_zaxis_range=[8,10],
    scene3_zaxis_range=[0,3],
    scene4_zaxis_range=[.05,.1],
    )

fig_equil2.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))

fig_equil2.show()

## Rotem washing LLC tests

### 6 - Washing1

In [62]:
EQUIL_rotem_washing3([[2.2,14.9,.0806]]).yout

array([[0.35013908, 1.81780071, 0.01405343]])